<a href="https://colab.research.google.com/github/vaishakh-v/ml_satellite_image_classification/blob/main/Satellite_Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# CubeSat

## Problem statement

The task at hand is to develop a machine learning model that accurately classifies data captured by CubeSats. The goal is to prioritize which images are most valuable for transmission back to Earth, given the limited onboard resources and slow data downlink speeds. The prioritization criteria involved dividing the data into five classes: Priority, Noisy, Blurry, Missing_data, and Corrupt.
The project is motivated by a nanosatelite mission, Visible Extra-galactic Background RadiaTion Exploration by CubeSat(VERTECS), a joing venture by the Kyushu institute of technology and collaborators. Due to size and resource constratints, the CubeSat was equiped with a prototype Camera Controller Beard(CCB) which carries a Raspberry Pi module 4 intended to run the machine learning model. The challenge therefore becomes finding the balance between a lightweight, fast inference model (usually simple models) and a high accuracy model(usually more comples and therefore heavier models) for classifying images received by the CubeSat.

More details on the hack4dev hackathon can be ffound through the following github link ` https://github.com/Hack4Dev/CubeSat_ImageClassify `.

### Solution techniques/strategies

#### Available tools

1. Clean data for training, validation and testing
2. Prewritten notebooks offering a framework to start from
3. A CubeSat CNN already developed with 100% accuracy

The problem is to ensure the model has:

i) High accuracy

ii) Fast inference speed(Low evaluation time)

ii) Low algorithm size

iii) Minimal strain on cpu(the Raspberry PI on which it is to run is limited 8GB RAM)

#### Heuristics

**1) Fine tune parameters to increase accuracy of simple models**
Applying Occam's razor principle of the simpler model should be the first one to investigate for a solution

**2) Reduce size and compute resources demanded by the CNN**
The CNN is already 100% accurate. Reducing size and evaluation time would suffice.

## Setting up the evaluation pipeline

In [ ]:
import threading
import psutil
import os
import time
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Function to monitor memory and CPU usage
def monitor_resources(mem_usage, cpu_usage, stop_event):
    process = psutil.Process(os.getpid())
    while not stop_event.is_set():
        mem = process.memory_info().rss / (1024 * 1024)  # Memory in MB
        cpu = process.cpu_percent(interval=None)  # CPU usage percentage
        mem_usage.append(mem)
        cpu_usage.append(cpu)
        time.sleep(0.1)  # Sampling interval

# Function to preprocess test data
def preprocess_data(preprocessing_fn, X_test_raw):
    return preprocessing_fn(X_test_raw)

# Function to make predictions
def make_predictions(model, X_test_processed):
    return model.predict(X_test_processed)

# Function to plot the confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.show()

def print_evaluation_results(metrics, class_names):
    print("\n### Evaluation Metrics ###\n")
    print(f"Evaluation Time:       {metrics['evaluation_time']:.2f} seconds")
    print(f"Peak Memory Usage:     {metrics['peak_memory_usage']:.2f} MB")
    print(f"Average CPU Usage:     {metrics['average_cpu_usage']:.2f} %")
    print(f"Algorithm code size:   {metrics['algorithm_code_size']:.3f} KB")
    print(f"Accuracy:              {metrics['accuracy']:.3f}")
    print(f"F1 Score:              {metrics['f1_score']:.3f}")
    print("\n### Confusion Matrix ###\n")
    plot_confusion_matrix(metrics['confusion_matrix'], class_names)

def compute_metrics(y_test, y_pred, class_names):
    metrics = {}
    if len(y_pred.shape) != 1:
        y_pred = np.argmax(y_pred, axis=1)
        y_test = np.argmax(y_test, axis=1)

    metrics['accuracy'] = accuracy_score(y_test, y_pred)
    metrics['f1_score'] = f1_score(y_test, y_pred, average='weighted')
    metrics['confusion_matrix'] = confusion_matrix(y_test, y_pred)
    return metrics

def calculate_algorithmCode_size(model, preprocessing_fn):
    # Model size handling
    if hasattr(model, 'get_model_size'):
        model_size = model.get_model_size()  # Already in MB
    else:
        model_size = len(pickle.dumps(model))/ (1024)  # Convert bytes to KB

    # Preprocessing size handling
    preprocessing_size = len(pickle.dumps(preprocessing_fn)) / (1024)  # Bytes to KB

    return round(model_size + preprocessing_size, 3)

def evaluate_pipeline(model, X_test_raw, y_test, preprocessing_fn):
    class_names = ["Blurry", "Corrupt", "Missing_Data", "Noisy", "Priority"]

    # Resource monitoring setup
    p = psutil.Process(os.getpid())
    p.cpu_affinity([2])
    mem_usage = []
    cpu_usage = []
    stop_monitoring = threading.Event()
    monitor_thread = threading.Thread(target=monitor_resources,
                                    args=(mem_usage, cpu_usage, stop_monitoring))
    monitor_thread.start()

    # Timing and processing
    start_time = time.time()
    X_test_processed = preprocess_data(preprocessing_fn, X_test_raw)
    y_pred = make_predictions(model, X_test_processed)
    end_time = time.time()

    # Cleanup and metrics collection
    stop_monitoring.set()
    monitor_thread.join()

    metrics = {
        'evaluation_time': end_time - start_time,
        'peak_memory_usage': max(mem_usage),
        'average_cpu_usage': np.mean(cpu_usage),
        'algorithm_code_size': calculate_algorithmCode_size(model, preprocessing_fn)
    }
    metrics.update(compute_metrics(y_test, y_pred, class_names))

    print_evaluation_results(metrics, class_names)

    # Memory cleanup
    del X_test_processed, y_pred
    gc.collect()

    return metrics
print("Done")

Done


## Importing stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle # saving models as a pickle file
import gc # garbage collector to free memory

#### XGBOOST specific imports

In [ ]:
import numpy as np
import os
from skimage.transform import resize
from xgboost import XGBClassifier
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import accuracy_score

#### CNN specific imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential  # Importing Sequential to build the model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import DepthwiseConv2D, SeparableConv2D
from keras.utils import to_categorical

## Reading the data and visualizing it

In the `data` folder, you will find three types of datasets, each saved as numpy files along with their corresponding label files. These datasets are organized as follows:

1.	`train_images.npy`: Contains images used for **training** machine and deep learning models. The associated labels are stored in train_labels.npy.
2.	`val_images.npy`: Contains images used for **validating** the trained models. The corresponding labels are stored in val_labels.npy.
3.	`test_images.npy`: Contains images used for **testing** the trained models. The associated labels are stored in test_labels.npy.

Let’s now read and explore these datasets.

In [ ]:
# Load the datasets
train_images = np.load('/content/drive/MyDrive/CubeSat-ML-Optimization-Dataset/test_images1.npy')
train_labels = np.load('/kaggle/input/cubesat-ml-optimization-dataeset/train_labels.npy')
val_images = np.load('/kaggle/input/cubesat-ml-optimization-dataeset/val_images.npy')
val_labels = np.load('/kaggle/input/cubesat-ml-optimization-dataeset/val_labels.npy')
test_images = np.load('/kaggle/input/cubesat-ml-optimization-dataeset/test_images1.npy')
test_labels = np.load('/kaggle/input/cubesat-ml-optimization-dataeset/test_labels.npy')

# Print basic information about each dataset
print(f"Training images: {train_images.shape}, Training labels: {train_labels.shape}")
print(f"Validation images: {val_images.shape}, Validation labels: {val_labels.shape}")
print(f"Testing images: {test_images.shape}, Testing labels: {test_labels.shape}")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/cubesat-ml-optimization-dataeset/train_images.npy'

- The training dataset consists of 9,711 samples of 512x512 RGB images, while the validation and testing sets each contain 3,237 samples.

### Visualising the data

The dataset we will be working with contains five classes, described as follows:
- **Blurry**: Data captured while the satellite is in motion, resulting in blurred images.
- **Corrupt**: Images with defects from improper camera priming or stray light.
- **Missing Data**: Images with partial or complete data loss.
- **Noisy**: Images over-saturated with noise from radiation or other sources.
- **Priority**: Clear images suitable for scientific analysis on the ground.

Now, Let’s take a look at these datasets.

In [ ]:
# Define the class names
class_names = ["Blurry", "Corrupt", "Missing_Data", "Noisy", "Priority"]

# Get the unique labels in the training set
unique_labels = np.unique(train_labels)


# Display the first 5 images for each class
for label in unique_labels:
    # Find the indices of images belonging to the current class
    class_indices = np.where(train_labels == label)[0]

    # Select the first 5 images of this class
    num_images_to_display = min(5, len(class_indices))
    selected_indices = class_indices[:num_images_to_display]
    selected_images = train_images[selected_indices] / 255.0  # Normalize images for better visualization

    # Plot the selected images
    fig, axes = plt.subplots(1, num_images_to_display, figsize=(20, 4))
    fig.suptitle(f'Class: {class_names[label]}', fontsize=16)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    for i, ax in enumerate(axes):
        ax.imshow(selected_images[i])
        ax.axis('off')

    plt.show()
    print()

If we were to rank these images in terms of importance based on their significance in capturing and transmitting them back to Earth, the order would be:

1.	`Priority`: Images with the highest importance and usability.
2.	`Noisy` & `Blurry`: Impure images that are potentially recoverable with preprocessing.
3.	`Corrupt` and `Missing Data`: Images with severe issues or missing information, making recovery or reuse least likely.

This ranking will help assess model performance by testing its ability to handle different levels of data quality and recover meaningful information from problematic images.

### Class Balance Check

In [ ]:
# Check the balance of the classes in each dataset
train_class_counts = np.bincount(train_labels)
val_class_counts = np.bincount(val_labels)
test_class_counts = np.bincount(test_labels)

# Display the class distribution with class names
print("\nClass distribution:")
print(f"Training set: {dict(zip(class_names, train_class_counts))}")
print(f"Validation set: {dict(zip(class_names, val_class_counts))}")
print(f"Testing set: {dict(zip(class_names, test_class_counts))}")

The `Priority` class has the most data, followed by `Noisy` and `Blurry`, while `Corrupt` has the least, indicating class imbalance.

## Stochastic Gradient Descent classifier

The SGD (Stochastic Gradient Descent) model was a model given at the start of the Hack4dev hackathon through this github link `https://github.com/Hack4Dev/CubeSat_ImageClassify/blob/main/3-ML.ipynb`.

This model's performance was poor and it was evidently insensible to forcefully try optimizing a linear classiefier to image data.

Moreover, the model performed very poorly on the chosen evaluation metrics.

We therefore dropped this model. Nonetheless, we deem it of importance to highlight it here.

## XGBOOST

XGBOOST was tried, motivated by Occam's razor principle to see if a simple model could be optimized to reach the desired accuracyy. The rationale behind this was that simpler model are more likely to have small algorithm size and exert less strain on the cpu.

### Preprocessing

The data was preprocessed using the `memory_safe_preprocessor()` function. The data is preprocessed in batches to minimise strain on the cpu.

In [ ]:
# Configuration
BATCH_SIZE = 512  # Process 512 images at a time
TARGET_SIZE = (64, 64)  # Reduced from 512*512 to save memory
PCA_COMPONENTS = 300

In [ ]:
def memory_safe_preprocessor(images, batch_size=BATCH_SIZE):
    """Process images in batches to avoid memory overload"""
    num_images = images.shape[0]
    for i in range(0, num_images, batch_size):
        batch = images[i:i+batch_size]

        # Process batch
        batch_pre = batch.astype('float32') / 255.0
        batch_pre = np.array([resize(img, TARGET_SIZE, anti_aliasing=True) for img in batch_pre])
        batch_pre = batch_pre.reshape(len(batch_pre), -1)  # Flatten

        # Clean up
        del batch
        gc.collect()
        yield batch_pre

In [ ]:
# Initialize Incremental PCA
ipca = IncrementalPCA(n_components=PCA_COMPONENTS, batch_size=BATCH_SIZE)

# Process training data in batches
train_batches = memory_safe_preprocessor(train_images)
X_train_pca = []

for i, batch in enumerate(train_batches):
    if i == 0:
        ipca.partial_fit(batch)
    X_batch_pca = ipca.transform(batch)
    X_train_pca.append(X_batch_pca)
    del batch, X_batch_pca
    gc.collect()

X_train_pca = np.concatenate(X_train_pca)


# Process validation data in batches
val_batches = memory_safe_preprocessor(val_images)
X_val_pca = []

for batch in val_batches:
    X_batch_pca = ipca.transform(batch)
    X_val_pca.append(X_batch_pca)
    del batch, X_batch_pca
    gc.collect()

X_val_pca = np.concatenate(X_val_pca)

### Model architecture definition

In [ ]:
# Handle class imbalance through sample weights
class_counts = np.bincount(train_labels)
class_weights = {i: sum(class_counts)/count for i, count in enumerate(class_counts)}
sample_weights = np.array([class_weights[lbl] for lbl in train_labels])

xgb_model = XGBClassifier(
    objective='multi:softmax',  # Explicitly set for multi-class
    num_class=5,
    n_estimators=250,          # Increase from 150 for increased accuracy
    learning_rate=0.05,        # Lower rate, more trees
    max_depth=7,               # Deeper trees (from 5)
    min_child_weight=3,         # Control overfitting
    gamma=0.2,                 # Regularization
    subsample=0.9,             # More data per tree
    colsample_bytree=0.8,
    tree_method='hist',
    reg_alpha=0.1,             # L1 regularization
    reg_lambda=0.1,            # L2 regularization
    eval_metric='mlogloss',    # Better for multi-class
    n_jobs=-1,
    random_state=42
)

### Training

In [ ]:
# Train in batches (manual incremental training)
batch_size = 2048
for i in range(0, len(X_train_pca), batch_size):
    xgb_model.fit(
        X_train_pca[i:i+batch_size],
        train_labels[i:i+batch_size],
        sample_weight=sample_weights[i:i+batch_size],
        xgb_model=xgb_model if i > 0 else None,  # Continue training
        eval_set=[(X_val_pca, val_labels)],
        verbose=0
    )
    print(f"Processed {min(i+batch_size, len(X_train_pca))}/{len(X_train_pca)} samples")
    gc.collect()

### Validation of trained model

In [ ]:
# Validate final model
val_preds = xgb_model.predict(X_val_pca)
val_acc = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_acc:.4f}")

# Save model components
import joblib
joblib.dump(ipca, 'pca_model.pkl')
joblib.dump(xgb_model, 'xgb_model.pkl')

The model achieves an accuracy of 67.47% much lower than the benchmarch cnn which had 100% accuracy.

Next we evaluate on the model to see how it performs on the chosen metrics....

### Evaluating the xgboost model

In [ ]:
#Preprocess Test Data (Same as Training)
def preprocess_test(images):
    """Identical preprocessing to training pipeline"""
    test_batches = memory_safe_preprocessor(images)
    processed = []
    for batch in test_batches:
        # Apply PCA transformation
        batch_pca = ipca.transform(batch)
        processed.append(batch_pca)
        del batch, batch_pca
        gc.collect()
    return np.concatenate(processed)

In [ ]:
evaluate_pipeline(xgb_model, test_images, test_labels, preprocess_test)

The model's evaluation results are too poor whether it is in algorithm size(15MB) to peak memory used (14GB - which by far exceeds our limit of 8GB).

The model performs worse than the benchmark.

## Convolutional Neural Networks

### Preprocessing

In [ ]:
def preprocessing_CNN(X):

    return X

In [ ]:
# One-hot encode the labels (assuming you have 5 classes)
train_labels = to_categorical(train_labels, num_classes=5)
val_labels = to_categorical(val_labels, num_classes=5)
test_labels_hot = to_categorical(test_labels, num_classes=5)

### Train CubeCatNet CNN mdoel

We will define and train a Convolutional Neural Network (CNN) model that was defined in the paper https://arxiv.org/pdf/2408.14865

In [ ]:
# Define the CNN model architecture
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(512, 512, 3)),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    Conv2D(32, (3, 3), activation='relu'),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    Conv2D(128, (3, 3), activation='relu'),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    GlobalAveragePooling2D(),  # Global average pooling layer
    Dense(5, activation='softmax')  # Output layer with 5 neurons (one for each class) + Softmax activation
])

# Compile the model with appropriate loss function, optimizer, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Model defined and compiled successfully.")

In [ ]:
model.summary()

#### Model  training

In [ ]:
# Train the model on the training data
history = model.fit(
    train_images, train_labels,
    epochs=10,  # Number of epochs
    batch_size=64,  # Batch size
)

print("Model training complete.")

##### **Saving the CNN model**

In [ ]:
import pickle

with open('cnn_model.pkl', 'wb') as file:
    pickle.dump(model, file)

#### Validation set results for CubeSat CNN

Opening the pickle file

In [ ]:
with open('cnn_model.pkl', 'rb') as file:
    cnn_loaded_model = pickle.load(file)

val_predictions = cnn_loaded_model.predict(val_images)

In [ ]:
# Ensure val_labels and val_predictions have the same samples (N)
print("val_labels shape:", val_labels.shape)
print("val_predictions shape:", val_predictions.shape)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert from one-hot or probability distributions to single integer class indices
val_labels = np.argmax(val_labels, axis=1)
val_predictions= np.argmax(val_predictions, axis=1)


# Detailed classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_predictions))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Define class names
class_names = ["Blurry", "Corrupt", "Missing_Data", "Noisy", "Priority"]

# Compute the confusion matrix
cm = confusion_matrix(val_labels, val_predictions)

# Plot the confusion matrix with class names
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

# Customize and display the plot
fig, ax = plt.subplots(figsize=(8, 8))  # Set the figure size
disp.plot(ax=ax, cmap='Blues', xticks_rotation='vertical')  # Use a blue colormap
plt.title("Confusion Matrix with Class Names")
plt.show()

#### Evaluating the model

In [ ]:
evaluate_pipeline(cnn_loaded_model, test_images, test_labels_hot, preprocessing_CNN)

This is the benchmark CNN model that is to be optimized.

### Implementing separable convo layers on the CubeSat CNN to reduce size

1. We use separable convo layers which reduce parameters by ~90% compared to the standard convolutions
2. We also use a depthwise+pointwise block motivated by the mobileNets architecture

In [ ]:
def create_compact_model():
    model = tf.keras.Sequential([
        SeparableConv2D(16, (3,3), activation='relu', input_shape=(512,512,3)),
        MaxPooling2D(2,2),
        SeparableConv2D(32, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        DepthwiseConv2D((3,3), activation='relu'),
        Conv2D(64, (1,1), activation='relu'),
        GlobalAveragePooling2D(),# Compile the model with appropriate loss function, optimizer, and metric
        Dense(5, activation='softmax')
    ])
    return model

# Compare model sizes
#original_model = model  # Your existing model
compact_model = create_compact_model()
compact_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model defined and compiled successfully.")

#print(f"Original Params: {original_model.count_params()/1e6:.1f}M")
print(f"Compact Params: {compact_model.count_params()/1e6:.1f}M")

In [ ]:
compact_model.summary()

`It is noteworthy that the params here have been reduced to 3,536 from 98,085 in the benchmark.`

In [ ]:
# Train the model on the training data
history = compact_model.fit(
    train_images, train_labels,
    epochs=10,  # Number of epochs
    batch_size=64,  # Batch size
)

print("Model training complete.")

#### saving the model

In [ ]:
import pickle

with open('cnn_model2.pkl', 'wb') as file:
    pickle.dump(compact_model, file)

#### Validation set results using the separable convo2D

In [ ]:
with open('cnn_model2.pkl', 'rb') as file:
    cnn_loaded_model2 = pickle.load(file)

val_predictions = cnn_loaded_model2.predict(val_images)

#### A lesson learned the hard way
1. Ensure val_labels and val_predictions have the same samples (N)
2. Verify whether or not they are one hot encoded to decide whether to do np.argmax.
3. Do np.argmax if and only if the shapes are 2D arrays. Keep in mind the np.argmax was done when evaluating the previous model. Reruning that line of code will intefere with data in the variable.


In [ ]:
print("val_labels shape:", val_labels.shape)
print("val_predictions shape:", val_predictions.shape)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert from one-hot or probability distributions to single integer class indices
val_predictions = np.argmax(val_predictions, axis=1)
#val_labels = np.argmax(val_labels, axis=1)

# Detailed classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_predictions))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Define class names
class_names = ["Blurry", "Corrupt", "Missing_Data", "Noisy", "Priority"]

# Compute the confusion matrix
cm = confusion_matrix(val_labels, val_predictions)

# Plot the confusion matrix with class names
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

# Customize and display the plot
fig, ax = plt.subplots(figsize=(8, 8))  # Set the figure size
disp.plot(ax=ax, cmap='Blues', xticks_rotation='vertical')  # Use a blue colormap
plt.title("Confusion Matrix with Class Names")
plt.show()

#### Evaluate on the compact_model

In [ ]:
evaluate_pipeline(compact_model, test_images, test_labels_hot, preprocessing_CNN)

The only caveat in this model's performance is the memory usage. Next we will try to reduce the peak memory used to under 8GB.

### Taking a gamble: trading accuracy for size and speed.

We drastically simplify the CNN to make it as light as possible.

The image sizes have also been reduced from (512*512) to (64 * 64). It is noteworthy to mention that the model accuracy is not interfered with in too much a way.

A challenge in laying out this architecture lay in tweaking the kernels, the number of separable convo layers and the degree of pooling.

In [ ]:
# the BEASHT-Net architecture
def compacter_model():
    model = tf.keras.Sequential([
        # Reduced filters + smaller kernel
        SeparableConv2D(8, (2,2), activation='relu', input_shape=(64,64,3)),

        MaxPooling2D(2,2),

        # Simplified block
        SeparableConv2D(8, (2,2), activation='relu'),
        #MaxPooling2D(2,2),

        # Removed depthwise layer (redundant computation)
        # Direct to final conv
        #Conv2D(16, (1,1), activation='relu'),

        GlobalAveragePooling2D(),
        Dense(5, activation='softmax')
    ])
    return model

`The commented layers in the architecture above was reached after the realization that with sufficient training, we could do without them. Furthermore, commenting them reduces drastically the model parameters and consequently, the model size from 40KB to 26KB and cpu usage from 6900MB to 6400MB.`

`The caveat here is that the model has to be trained for much longer. Moving from the previous separable_convo cnn, we had to train for 150 epochs for comparable performance. For this modified BEASHT_Net architecture, training will be done for 400 epochs.`

In [ ]:
malice_model = compacter_model()
malice_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model defined and compiled successfully.")

#print(f"Original Params: {original_model.count_params()/1e6:.1f}M")
print(f"Compact Params: {malice_model.count_params()/1e6:.1f}M")

In [ ]:
malice_model.summary()

`It is noteworthy that the params here have been reduced to 193 from the compact_model's 3,536 and from the 98,085 params in the benchmark.`

`The prev model` - After a few experiments with the model always converging at 98.97% accuracy, a decision to "overtrain" the model was reached. To push the number of epoch to 150 and study how that affects the model.
`Modified architecture` - we will instead train for 400 epochs rather than 150...training overdriveee...

In [ ]:
# Resize images to 128x128 (if originally 512x512)
train_images = tf.image.resize(train_images, [64, 64]).numpy()

In [ ]:
# If you have validation/test data:
val_images = tf.image.resize(val_images, [64, 64]).numpy()

In [ ]:
# Then train with:
history = malice_model.fit(
    train_images, train_labels,
    epochs=400, # increase to 1000 from 400 ....why?? why not ha ha?? maybe it will increase accuracy from 99.5 to 99.99 lol ha ha ha
    batch_size=64,
    #validation_data=(val_images, val_labels)  # If using validation
)

In [ ]:
import matplotlib.pyplot as plt

# Create subplots
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], color='orange', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_progress.png')  # Save to file
plt.show()

#### Saving the model

In [ ]:
import pickle

with open('malice.pkl', 'wb') as file:
    pickle.dump(malice_model, file)

#### Validation set results for the malice model

In [ ]:
with open('malice.pkl', 'rb') as file:
    malice_model = pickle.load(file)

In [ ]:
val_predictions = malice_model.predict(val_images)

In [ ]:
print("val_labels shape:", val_labels.shape)
print("val_predictions shape:", val_predictions.shape)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert from one-hot or probability distributions to single integer class indices
val_pred_classes = np.argmax(val_predictions, axis=-1)
#val_labels = np.argmax(val_labels, axis=1)


# Detailed classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_pred_classes))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Define class names
class_names = ["Blurry", "Corrupt", "Missing_Data", "Noisy", "Priority"]

# Compute the confusion matrix
cm = confusion_matrix(val_labels, val_pred_classes)

# Plot the confusion matrix with class names
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

# Customize and display the plot
fig, ax = plt.subplots(figsize=(8, 8))  # Set the figure size
disp.plot(ax=ax, cmap='Blues', xticks_rotation='vertical')  # Use a blue colormap
plt.title("Confusion Matrix with Class Names")
plt.show()

### Evaluation on the malice model

In [ ]:
test_images_red = tf.image.resize(test_images, [64,64]).numpy()

In [ ]:
evaluate_pipeline(malice_model, test_images_red, test_labels_hot, preprocessing_CNN)

### Quantizing the malice model using tensorflow lite

In [ ]:
def quantize_model(keras_model, save_path='quantized_model.tflite'):
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    with open(save_path, 'wb') as f:
        f.write(tflite_model)

quantize_model(malice_model) #run once to quantize

### Preparation for evaluation

The below QuantizeWrapper is a helper class used to package the tflite model(quantized model) in order for evaluation in the provided pipeline. The provided pipeline does pickling(takes models as pickle files) which is not native form of tflite. More specifically, the quantize wrapper creates a TensorFlow Lite XNNPACK delegate for CPU whose size we can determine by the get_model_size method in the wrapper calss..

In [ ]:
class QuantizedWrapper:
    def __init__(self, tflite_path):
        self.tflite_path = tflite_path
        self.interpreter = tf.lite.Interpreter(model_path=tflite_path)
        self.interpreter.allocate_tensors()
        self.input_index = self.interpreter.get_input_details()[0]['index']
        self.output_index = self.interpreter.get_output_details()[0]['index']

    def predict(self, X):
        outputs = []
        for x in X:
            x = x.astype('float32')
            self.interpreter.set_tensor(self.input_index, [x])
            self.interpreter.invoke()
            outputs.append(self.interpreter.get_tensor(self.output_index)[0])
        return np.array(outputs)

    def get_model_size(self):
        """Returns size in MB with verification"""
        if not os.path.exists(self.tflite_path):
            return 0.0
        bytes_size = os.path.getsize(self.tflite_path)
        return round(bytes_size / (1024), 4)  # KB with 4 decimals

    def __getstate__(self):
        state = self.__dict__.copy()
        del state['interpreter']
        return state

    def __setstate__(self, state):
        self.__dict__.update(state)
        self.interpreter = tf.lite.Interpreter(model_path=self.tflite_path)
        self.interpreter.allocate_tensors()
        self.input_index = self.interpreter.get_input_details()[0]['index']
        self.output_index = self.interpreter.get_output_details()[0]['index']

In [ ]:
# In evaluation pipeline:
q_model = QuantizedWrapper('quantized_model.tflite')

### Validation on the quantized model

In [ ]:
val_predictions = q_model.predict(val_images)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert from one-hot or probability distributions to single integer class indices
val_pred_classes = np.argmax(val_predictions, axis=-1)
#val_labels = np.argmax(val_labels, axis=1)


# Detailed classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_pred_classes))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Define class names
class_names = ["Blurry", "Corrupt", "Missing_Data", "Noisy", "Priority"]

# Compute the confusion matrix
cm = confusion_matrix(val_labels, val_pred_classes)

# Plot the confusion matrix with class names
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

# Customize and display the plot
fig, ax = plt.subplots(figsize=(8, 8))  # Set the figure size
disp.plot(ax=ax, cmap='Blues', xticks_rotation='vertical')  # Use a blue colormap
plt.title("Confusion Matrix with Class Names")
plt.show()

We can verify above that performance is exactly like the initial malice_model that was quantized.

In [ ]:
evaluate_pipeline(q_model, test_images_red, test_labels_hot, preprocessing_CNN)

___

#### Evaluation was done in the notebook delegated by the hackathon organizers

##### **⚠️ Freeing up Space**

In [ ]:
import gc

# Since we will no longer need the original training data (train_images), we can remove it from memory
del val_predictions, val_labels, val_images

# Force garbage collection to free up memory
gc.collect()

%reset -f
print("Data removed from memory.")